In [ ]:
'''
1. This is a basic counter for acconting how many MOFs with a certain chemical symbols exist in my database.
2. This is a effective way to construct a csv file which contain all MOFs I choose
3. This is a workchain to collect experimental isotherms for my database
'''

In [2]:
import numpy as np
# generate isotherms with pressure in bar and loading in cm^3/g, rank the isotherm from low pressures to high.
def transfer_isotherms_unit(inputfile, outputfile, pressure, loading):
    data = np.loadtxt(inputfile, delimiter=',')
    if pressure == "torr":
        Transfer_unit1 = 1/760
    elif pressure == "bar":
        Transfer_unit1 = 1
    elif pressure == "Pa":
        Transfer_unit1 = 1/1e5
    else:
        raise ValueError("The pressure unit is not supported")
    if loading == "wt%":# only work for CO2
        Transfer_unit2 = 1000*22.4/44.01/100
    elif loading == "cm^3/g":
        Transfer_unit2 = 1
    elif loading == "mol/kg":
        Transfer_unit2 = 22.4
    else:
        raise ValueError("The loading unit is not supported")
    data[:,0] = data[:,0]*Transfer_unit1
    data[:,1] = data[:,1]*Transfer_unit2
    indices = np.argsort(data[:, 0])
    data = data[indices]
    np.savetxt(outputfile, data, delimiter=',')

path0 = "/home/yutao/project/InN-MOF/ZIDDIB/293K_origin.csv"
path1 = "/home/yutao/project/InN-MOF/ZIDDIB/293K_short.csv"
#transfer_isotherms_unit(path0, path1, "torr", "wt%")
transfer_isotherms_unit(path0, path1, "bar", "cm^3/g")

In [ ]:
# this part is in order to generate the RSM list for the selected element
import os
import subprocess
import pandas as pd

def createdict(outputcsv, outputdir, datadir, filename):
    


In [ ]:
# this part is in order to generate the RSM list for the selected element
import os
import subprocess
import pandas as pd
picked_element = 'Ga'
outputpath = f"/home/yutao/dataset/RSM_{picked_element}.csv"
outputdir = f"/home/yutao/project/{picked_element}-MOF"
path = "/home/yutao/dataset/RSM_database/RSM_database/cifs"
data = pd.read_csv(outputpath)

# Check if the directory exists
if not os.path.exists(outputdir):
    # If not, create it
    os.makedirs(outputdir)
    print(f"Create {outputdir}")

for index, choice in enumerate(data['Usage']):
    if isinstance(choice, str)  and 'yes' in choice.lower():
        synonyms = data['Synonyms'][index]
        if isinstance(synonyms, str)  and 'no' not in synonyms.lower():
            dirname = synonyms
        else:
            dirname = data['CSD_label'][index]
        structure_dir = os.path.join(outputdir, dirname)
        if not os.path.exists(structure_dir):
            print(f"Create {structure_dir}")
            os.makedirs(structure_dir)
            cif_file = os.path.join(path,data['cif.label'][index]+".cif")
            subprocess.run(["cp", cif_file, structure_dir])


In [ ]:
def read_mof_csv(file_path):
    '''
    Read the csv file to get the mof information
    '''
    import pandas as pd
    mof_df = pd.read_csv(file_path)
    return mof_df

In [ ]:
# this part is in order to generate the RSM list for the selected element

import pandas as pd
from ase.io import read, write
import os

picked_element = 'Al'
outputpath = f"/home/yutao/dataset/RSM_{picked_element}_functional.csv"
path = "/home/yutao/dataset/RSM_database/RSM_database/cifs"
filepath = "/home/yutao/project/MIL-120/RSM_database/RSM_database/all_mof_features.csv"
data = pd.read_csv(filepath)
chemical_formulas = []
idx_ls = []
for index, rsm_name in enumerate(data['cif.label']):
    cif_file = os.path.join(path,rsm_name+".cif")
    if not os.path.exists(cif_file):
        continue
    atoms = read(cif_file)
    if picked_element in atoms.symbols and len(set(atoms.symbols)) == 5 and 'N' not in atoms.symbols:
        formula_string = atoms.get_chemical_formula()
        chemical_formulas.append(formula_string)
        idx_ls.append(index)

selected_data = data.loc[idx_ls, ['cif.label', 'MOFname']]
csd_names = [name.split('_')[0] for name in selected_data['MOFname']]
selected_data['CSD_label'] = csd_names
selected_data['Formula'] = chemical_formulas
if os.path.exists(outputpath):
    print(f"{outputpath} exists")
else:
    selected_data.to_csv(outputpath, index=False)
    print(f"{outputpath} is created")

# this part is in order to generate the RSM list for the selected element

import pandas as pd
from ase.io import read, write
import os

outputpath = f"/home/yutao/dataset/RSM_{picked_element}_N.csv"
path = "/home/yutao/dataset/RSM_database/RSM_database/cifs"
filepath = "/home/yutao/project/MIL-120/RSM_database/RSM_database/all_mof_features.csv"
data = pd.read_csv(filepath)
chemical_formulas = []
idx_ls = []
for index, rsm_name in enumerate(data['cif.label']):
    cif_file = os.path.join(path,rsm_name+".cif")
    if not os.path.exists(cif_file):
        continue
    atoms = read(cif_file)
    if picked_element in atoms.symbols and len(set(atoms.symbols)) == 5 and 'N' not in atoms.symbols:
        formula_string = atoms.get_chemical_formula()
        chemical_formulas.append(formula_string)
        idx_ls.append(index)

selected_data = data.loc[idx_ls, ['cif.label', 'MOFname']]
csd_names = [name.split('_')[0] for name in selected_data['MOFname']]
selected_data['CSD_label'] = csd_names
selected_data['Formula'] = chemical_formulas
if os.path.exists(outputpath):
    print(f"{outputpath} exists")
else:
    selected_data.to_csv(outputpath, index=False)
    print(f"{outputpath} is created")

In [ ]:
import pandas as pd
for picked_element in ['In']:
    finished_path = f"/home/yutao/dataset/RSM_{picked_element}_functional.csv"
    new_path = f"/home/yutao/dataset/CoRE_{picked_element}_functional.csv"
    outputpath = f"/home/yutao/dataset/extra_{picked_element}_functional.csv"
    data0 = pd.read_csv(finished_path)
    finish_ls = list(data0['CSD_label'])
    data = pd.read_csv(new_path)
    idx_ls = []
    for index, csd_name in enumerate(data['CSD_label']):
        if csd_name not in finish_ls:
            idx_ls.append(index)
        else:
            print("delete one structure I have already searched")

    selected_data = data.loc[idx_ls, ['MOFname', 'CSD_label', 'Formula']]
    selected_data.to_csv(outputpath, index=False)